# 平均報酬におけるUCRL2アルゴリズム

参考：
* [Near-optimal Regret Bounds for Reinforcement Learning](https://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)

今回は平均報酬でMinimax regretを達成するアルゴリズムを学びます．

表記：

* アルゴリズム$\mathfrak{U}$について，累積報酬：$R(M, \mathfrak{A}, s, T):=\sum_{t=1}^T r_t$
* 平均報酬：$\rho(M, \mathfrak{A}, s):=\lim _{T \rightarrow \infty} \frac{1}{T} \mathbb{E}[R(M, \mathfrak{A}, s, T)]$
* MDPの半径：$D(M):=\max _{s, s^{\prime} \in \mathcal{S}} \min _{\pi: \mathcal{S} \rightarrow \mathcal{A}} \mathbb{E}\left[T\left(s^{\prime} \mid M, \pi, s\right)\right]$.
    * 有限な半径のMDPを特にCommunicatingと呼びます．
    * このとき，最適な平均報酬は状態に依存しません：$\rho^*(M):=\rho^*(M, s):=\max _\pi \rho(M, \pi, s)$.
* リグレット：$\Delta(M, \mathfrak{A}, s, T):=T \rho^*(M)-R(M, \mathfrak{A}, s, T)$


## UCRL2アルゴリズム

![UCRL2](figs/UCRL2.png)

UCRL2アルゴリズムではExtended value iterationアルゴリズムを走らせるので，先にこちらを紹介します．

### Extended Value Iterationアルゴリズム

* $u_i(s)$を$i$イテレーション目の状態の価値とします．
* $u'_i(s)$をnormalizeされた価値とします

Extended value iterationは次を繰り返します．

$$
\begin{aligned}
u_0(s) & =0, \\
u_{i+1}(s) & =\max _{a \in \mathcal{A}}\left\{\tilde{r}_k(s, a)+\max _{p(\cdot) \in \mathcal{P}(s, a)}\left\{\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime}\right) \cdot u_i\left(s^{\prime}\right)\right\}\right\}
\end{aligned}
$$

ここで，$\tilde{r}_k$は下の集合の内最大のもの，$\mathcal{P}$は下の集合そのものを表します．
$$
\begin{aligned}
\left|\tilde{r}(s, a)-\hat{r}_k(s, a)\right| & \leq \sqrt{\frac{7 \log \left(2 S A t_k / \delta\right)}{2 \max \left\{1, N_k(s, a)\right\}}} \\
\left\|\tilde{p}(\cdot \mid s, a)-\hat{p}_k(\cdot \mid s, a)\right\|_1 & \leq \sqrt{\frac{14 S \log \left(2 A t_k / \delta\right)}{\max \left\{1, N_k(s, a)\right\}}} .
\end{aligned}
$$

$\max_p$は線形計画法で解くことができます．
この価値反復法は
$$
\max _{s \in S}\left\{u_{i+1}(s)-u_i(s)\right\}-\min _{s \in S}\left\{u_{i+1}(s)-u_i(s)\right\}<\frac{1}{\sqrt{t_k}}
$$
で停止します．

## UCRL2の解析

UCRL2のリグレットを導出します．
$k$エピソード目のリグレットを

$$
\Delta_k:=\sum_{s, a} \nu_k(s, a)\left(\rho^*-\bar{r}(s, a)\right)
$$

として定義しましょう．ここで，$\nu_k$はエピソード内で$(s, a)$を訪れた回数です．

---

このとき，次が高確率で成立します．

$$
\sum_k \Delta_k+\sqrt{\frac{5}{2} T \log \left(\frac{8 T}{\delta}\right)}
$$

**証明**

Hoeffdingの不等式から，$T$ステップ間の状態行動のカウント$N(s, a)$に対して，
$$
\mathbb{P}\left\{\left.\sum_{t=1}^T r_t \leq \sum_{s, a} N(s, a) \bar{r}(s, a)-\sqrt{\frac{5}{8} T \log \left(\frac{8 T}{\delta}\right)} \right\rvert\,(N(s, a))_{s, a}\right\} \leq\left(\frac{\delta}{8 T}\right)^{5 / 4}<\frac{\delta}{12 T^{5 / 4}}
$$

が成立します（5は多分$T^{5/4}$でlog取ったら出てくる）．
よって，
$$
\Delta\left(s_1, T\right)=T \rho^*-\sum_{t=1}^T r_t<T \rho^*-\sum_{s, a} N(s, a) \bar{r}(s, a)+\sqrt{\frac{5}{8} T \log \left(\frac{8 T}{\delta}\right)}
$$
が確率$1-\frac{\delta}{12 T^{5 / 4}}$以上で成立します．
後は書き直せば終わりです．

---



期待リグレットを出す場合は真のMDPが$\mathfrak{M}$に入っていない場合のリグレットを出す必要がありますが，面倒なので省略します．

### $M \in \mathfrak{M}_k$の場合のリグレット　

Optimismは後で示します．
先にOptimismが成立している場合のリグレットを出してしまいましょう．
まず，Extended Value Iterationの仕組みから，

$$
\tilde{\rho}_k \geq \rho^*-\frac{1}{\sqrt{t_k}}
$$

が成立します．よって，
$$
\Delta_k \leq \sum_{s, a} v_k(s, a)\left(\rho^*-\bar{r}(s, a)\right) \leq \sum_{s, a} v_k(s, a)\left(\tilde{\rho}_k-\bar{r}(s, a)\right)+\sum_{s, a} \frac{v_k(s, a)}{\sqrt{t_k}}
$$

が成り立ちます．
